# Démonstration : One-Time Pad (OTP)

Ce notebook implémente et explore le One-Time Pad, seul système prouvé parfaitement sécurisé.

**Objectifs** :
- Implémenter OTP avec XOR
- Vérifier la propriété de correction
- Démontrer expérimentalement l'uniformité de la distribution
- Montrer la catastrophe de la réutilisation de clé (Two-Time Pad attack)

In [ ]:
import os
import secrets
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

## 1. Implémentation de base

In [ ]:
def otp_gen(n: int) -> bytes:
    """
    Génère une clé aléatoire de n bytes.
    Utilise secrets.token_bytes() qui est cryptographiquement sécurisé.
    """
    return secrets.token_bytes(n)

def otp_encrypt(message: bytes, key: bytes) -> bytes:
    """
    Chiffre un message avec OTP.
    
    Args:
        message: Message à chiffrer (bytes)
        key: Clé OTP (même longueur que message)
    
    Returns:
        Ciphertext (bytes)
    """
    if len(message) != len(key):
        raise ValueError("Message et clé doivent avoir la même longueur !")
    
    # XOR byte par byte
    return bytes(m ^ k for m, k in zip(message, key))

def otp_decrypt(ciphertext: bytes, key: bytes) -> bytes:
    """
    Déchiffre un ciphertext avec OTP.
    Identique à l'encryption (XOR est son propre inverse) !
    """
    return otp_encrypt(ciphertext, key)  # XOR est involutif

## 2. Test de correction

In [ ]:
# Message original
message = b"HELLO WORLD! This is a secret message."
print(f"Message original : {message}")
print(f"Longueur : {len(message)} bytes")

# Génération de clé
key = otp_gen(len(message))
print(f"\nClé (hex) : {key.hex()}")

# Chiffrement
ciphertext = otp_encrypt(message, key)
print(f"\nCiphertext (hex) : {ciphertext.hex()}")
print(f"Ciphertext (incompréhensible) : {ciphertext}")

# Déchiffrement
decrypted = otp_decrypt(ciphertext, key)
print(f"\nMessage déchiffré : {decrypted}")

# Vérification
assert message == decrypted, "Erreur : déchiffrement incorrect !"
print("\n✅ Test de correction réussi !")

## 3. Uniformité de la distribution

**Théorie** : Pour tout message fixe $m$, si la clé $k$ est uniforme sur $\{0,1\}^n$, alors $c = m \oplus k$ est aussi uniforme sur $\{0,1\}^n$, **indépendamment de $m$** !

**Expérience** : On fixe un message, on génère beaucoup de clés aléatoires, et on regarde la distribution des ciphertexts.

In [ ]:
# Fixe un message d'un byte
message_byte = b"A"  # 0x41 = 01000001 en binaire

# Génère 10000 chiffrements avec clés aléatoires
num_trials = 10000
ciphertexts = []

for _ in range(num_trials):
    key = otp_gen(1)
    c = otp_encrypt(message_byte, key)
    ciphertexts.append(c[0])  # Récupère le byte

# Compte la fréquence de chaque valeur (0-255)
counts = Counter(ciphertexts)

# Visualisation
plt.figure(figsize=(12, 5))
plt.bar(counts.keys(), counts.values(), color='skyblue', alpha=0.7)
plt.axhline(y=num_trials/256, color='r', linestyle='--', label=f'Uniforme attendu: {num_trials/256:.1f}')
plt.xlabel('Valeur du ciphertext byte (0-255)')
plt.ylabel('Fréquence')
plt.title(f'Distribution des ciphertexts OTP\n(Message fixe = "A", {num_trials} essais)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"\n📊 Statistiques :")
print(f"Fréquence attendue (uniforme) : {num_trials/256:.2f} par valeur")
print(f"Fréquence moyenne observée : {np.mean(list(counts.values())):.2f}")
print(f"Écart-type : {np.std(list(counts.values())):.2f}")
print(f"\n✅ La distribution est approximativement uniforme !")

## 4. Two-Time Pad Attack : La catastrophe de la réutilisation

**Scénario** : Alice réutilise la même clé $k$ pour chiffrer deux messages $m_1$ et $m_2$ :
$$
c_1 = m_1 \oplus k \\
c_2 = m_2 \oplus k
$$

**Attaque** : Eve calcule :
$$
c_1 \oplus c_2 = (m_1 \oplus k) \oplus (m_2 \oplus k) = m_1 \oplus m_2
$$

Eve obtient le XOR des deux messages ! Si les messages sont en anglais, elle peut exploiter les statistiques linguistiques pour retrouver $m_1$ et $m_2$.

In [ ]:
# Deux messages secrets
message1 = b"ATTACK AT DAWN"
message2 = b"RETREAT TONIGHT"

# Alice réutilise la même clé (ERREUR FATALE !)
key_reused = otp_gen(max(len(message1), len(message2)))

# Padding des messages à la même longueur
message1_padded = message1 + b" " * (len(message2) - len(message1))

# Chiffrement
c1 = otp_encrypt(message1_padded, key_reused[:len(message1_padded)])
c2 = otp_encrypt(message2, key_reused[:len(message2)])

print("Alice chiffre deux messages avec la MÊME clé :")
print(f"Message 1 : {message1_padded}")
print(f"Message 2 : {message2}")
print(f"\nCiphertext 1 (hex) : {c1.hex()}")
print(f"Ciphertext 2 (hex) : {c2.hex()}")

# Eve intercepte c1 et c2
# Elle calcule c1 ⊕ c2 = m1 ⊕ m2
xor_messages = bytes(a ^ b for a, b in zip(c1, c2))

print(f"\n🔓 Eve calcule c1 ⊕ c2 :")
print(f"Résultat (hex) : {xor_messages.hex()}")
print(f"Résultat (ASCII) : {xor_messages}")
print(f"\n⚠️  Eve a obtenu m1 ⊕ m2 = {xor_messages}")
print(f"\nAvec analyse linguistique (fréquences, patterns anglais),")
print(f"Eve peut retrouver m1 et m2 ! Voir exercices pour implémentation.")

# Vérification
expected_xor = bytes(a ^ b for a, b in zip(message1_padded, message2))
assert xor_messages == expected_xor
print(f"\n❌ JAMAIS réutiliser une clé OTP !")

## 5. Exemple historique : Projet VENONA

Entre 1943 et 1980, la NSA a décrypté des milliers de messages soviétiques chiffrés avec OTP grâce à des **réutilisations de clés**.

**Contexte** : Pendant la Seconde Guerre mondiale, l'URSS manquait de clés OTP et a réutilisé certains pads.

**Résultat** : Exposition de réseaux d'espionnage, identification d'agents (dont Klaus Fuchs du projet Manhattan).

**Leçon** : Même avec OTP "parfaitement sécurisé", les erreurs opérationnelles sont fatales !

## 6. Exercices

### Exercice 1 : Malléabilité

Le OTP est **malléable** : un attaquant peut modifier le ciphertext de manière contrôlée sans connaître la clé.

**Question** : Supposons qu'Eve intercepte $c = \text{Enc}_k(\text{"TRANSFER \\$1000"})$. Comment peut-elle modifier $c$ pour que le déchiffrement donne "TRANSFER \$9999" ?

*Indice* : Calculer le XOR nécessaire et l'appliquer à $c$.

### Exercice 2 : Two-Time Pad avec texte anglais

Implémenter une attaque complète sur deux ciphertexts chiffrés avec la même clé, en utilisant :
- Fréquences des lettres anglaises
- Patterns de mots communs (THE, AND, etc.)
- Espaces (très révélateurs !)

### Exercice 3 : Sécurité avec clé plus courte

Implémenter et analyser la variante : $\text{Enc}_k(m) = m \oplus (k \| k)$ où la clé $k$ est répétée.

Montrer qu'elle n'a PAS la sécurité parfaite.

## Conclusion

**Points clés** :
- OTP est simple et parfaitement sécurisé
- XOR a des propriétés cryptographiques idéales
- Distribution uniforme prouvée expérimentalement
- ❌ Réutilisation de clé = catastrophe totale
- ❌ Clé doit être aussi longue que le message
- ❌ Pas d'intégrité (attaquant peut modifier)

En pratique, OTP est rarement utilisé. Le chapitre suivant introduira les **chiffrements par flot** (stream ciphers) qui relaxent les contraintes.